# Overview

This notebooks demonstrates how to perform pin drop sampling using barangays as PSUs for a single province in the Philippines. 

- you also need to download rooftop data --> mention that this has already been downloaded and the location and the issues with downloading it again from the VIDA website

- 


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
from pathlib import Path
import pandas as pd
from datetime import datetime
import folium
from tqdm import tqdm
from pin_drop_sampling2.utils import get_s2_cell_id
from pin_drop_sampling2.utils import count_neighbors_in_radius

# Set file locations
Set the location of the file with the PSU boundaries and population counts and the directory for the rooftop data files below.

In [3]:
DB_DIR = Path.home() / 'IDinsight Dropbox' / 'Random Walk Testing' 
PSU_FILE = DB_DIR / '01_Raw data'/ '03_Census' / 'Philippines' / 'barangay_w_borders.parquet'
ROOFTOP_DIR = DB_DIR /'01_Raw data'/ '01_Rooftop'/'Philippines'
OUTPUT_DIR = DB_DIR / '03_Output' / '05_HPLS qual'
psus = gpd.read_parquet(PSU_FILE)
psus.head()

,PSGC,brgy_name,GeographicLevel,brgy_urbanrural,brgy_pop,Status,reg_code,prov_code,citymun_code,reg_name,...,adm1_psgc,adm2_psgc,adm3_psgc,adm4_en,geo_level,len_crs,area_crs,len_km,area_km2,geometry
0,402103091,Talaba 3,Bgy,-,7464,,0400000000,0402100000,0402103000,Region IV-A (CALABARZON),...,400000000.0,402100000.0,402103000.0,Talaba 3,Bgy,2390.0,195646.0,2.0,0.0,"POLYGON ((120.96283 14.46184, 120.96241 14.461..."
1,402103092,Zapote 1,Bgy,-,10983,,0400000000,0402100000,0402103000,Region IV-A (CALABARZON),...,400000000.0,402100000.0,402103000.0,Zapote 1,Bgy,1680.0,135998.0,1.0,0.0,"POLYGON ((120.9642 14.46274, 120.96384 14.4625..."
2,402103093,Zapote 2,Bgy,-,4331,,0400000000,0402100000,0402103000,Region IV-A (CALABARZON),...,400000000.0,402100000.0,402103000.0,Zapote 2,Bgy,2192.0,118941.0,2.0,0.0,"POLYGON ((120.96686 14.46325, 120.96723 14.462..."
3,402103087,Real,Bgy,-,9981,,0400000000,0402100000,0402103000,Region IV-A (CALABARZON),...,400000000.0,402100000.0,402103000.0,Real,Bgy,5169.0,754994.0,5.0,0.0,"POLYGON ((120.94616 14.4364, 120.94737 14.4365..."
4,402103077,Aniban 2,Bgy,-,5180,,0400000000,0402100000,0402103000,Region IV-A (CALABARZON),...,400000000.0,402100000.0,402103000.0,Aniban 2,Bgy,2008.0,179642.0,2.0,0.0,"POLYGON ((120.96776 14.46004, 120.96794 14.459..."


# Sample PSUs

The following code samples 20 barangays per PSU (if there are more than 20 barangays). This code is mainly for demo purposes. I'm not 100% sure that this code works. PPS sampling can be surprisingly tricky and thus we might want to perform this step in Stata or R. We would also want to stratify by urban/rural status when we do this for real.

In [5]:
num_brgys_per = 3

def pps_sample(group):
    # Normalize the weights for the group
    probabilities = group['brgy_pop'] / group['brgy_pop'].sum()
    
    num_to_sample = min(num_brgys_per, group.shape[0])

    # Sample without replacement using the normalized weights
    sampled_group = group.sample(n=num_to_sample, weights=probabilities, replace=False)
    return sampled_group

sampled_barangays = psus.groupby('prov_code', group_keys=False).apply(pps_sample)
# save the sampled barangays
timestamp = datetime.now().strftime("%Y%m%d_%H")
sampled_barangays.to_parquet(OUTPUT_DIR / f'samp_bars_{timestamp}.parquet')

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_5677/851008615.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_barangays = psus.groupby('prov_code', group_keys=False).apply(pps_sample)


In [12]:
barangays_missing_geometry = sampled_barangays[sampled_barangays['geometry'].isna()]
print(f'There are {len(barangays_missing_geometry)} barangays with missing geometry')

# drop barangays with missing geometry
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
sampled_barangays = sampled_barangays.dropna(subset=['geometry'])

There are 0 barangays with missing geometry


# Generate dataset of rooftops in sampled barangays
The code below filters the rooftop data and generates a single dataset with only the rooftops within the sampled barangays. Note that a) this can take quite a bit of time and b) if any barangays happen to straddle more than one s2 cell only a portion of rooftops will be included. This is 

In [34]:
# get the s2 cell id for each barangay
sampled_barangays['s2_cell_id'] = sampled_barangays.apply(lambda x: get_s2_cell_id(x.geometry.centroid, 4), axis=1)

# create empty gdf to store rooftops
all_rooftops = gpd.GeoDataFrame()

# loop over each unique value of s2_cell_id. I loop over s2_cell_id first because loading the rooftop data for each s2 cell 
# takes a lot of time so I want to do it only once for each s2 cell
pd.options.mode.chained_assignment = None  # turn off annoying copy of a df warning
for s2_cell_id in sampled_barangays['s2_cell_id'].unique():
    print(f"\nProcessing s2 cell {s2_cell_id}")
    # get the barangays in this s2 cell
    barangays_in_s2_cell= sampled_barangays[sampled_barangays['s2_cell_id'] == s2_cell_id]
    # load the rooftop data for this s2 cell
    rooftops_gdf = gpd.read_parquet(ROOFTOP_DIR /f'{s2_cell_id}.parquet')
    # replace geometry column with the centroid of the geometry
    rooftops_gdf['geometry'] = rooftops_gdf.geometry.centroid

    for item, row in barangays_in_s2_cell.iterrows():
        # print a single dot without the newline character
        print('.', end='')
        # filter rooftops to only include those within the barangay
        temp_rooftops = rooftops_gdf[rooftops_gdf.geometry.within(row.geometry)]
        # set the psid for the rooftops
        temp_rooftops['PSGC'] = row['PSGC']
        temp_rooftops.to_crs(epsg=4326, inplace=True)
        # try to append temp_rooftops to all_rooftops and catch a value error. if there is an error, print the crs of the two dataframes
        try:
            all_rooftops = gpd.GeoDataFrame(pd.concat([all_rooftops, temp_rooftops], ignore_index=True))
        except ValueError:
            print(f"Error with s2 cell {s2_cell_id}")
            print(f"temp_rooftops crs: {temp_rooftops.crs}")
            print(f"all_rooftops crs: {all_rooftops.crs}")
            
# save all rooftops to a parquet file in case I close this notebook or the kernel gets messed up
all_rooftops.to_parquet(OUTPUT_DIR / f'all_roofs_samp_bars_{timestamp}.parquet')


Processing s2 cell 3715469692580659200


/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_1157/345339898.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rooftops_gdf['geometry'] = rooftops_gdf.geometry.centroid


........................................................................................................................................
Processing s2 cell 3778520087363846144


/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_1157/345339898.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rooftops_gdf['geometry'] = rooftops_gdf.geometry.centroid


...
Processing s2 cell 3724476891835400192


...

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_1157/345339898.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rooftops_gdf['geometry'] = rooftops_gdf.geometry.centroid


............................................
Processing s2 cell 3616390500778508288


/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_1157/345339898.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rooftops_gdf['geometry'] = rooftops_gdf.geometry.centroid


.
Processing s2 cell 3733484091090141184
.

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_1157/345339898.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  rooftops_gdf['geometry'] = rooftops_gdf.geometry.centroid


# Identify isolated rooftops
We identify and filter out isolated rooftops with no other rooftops around. These rooftops may not have people living in or near them and could result in very high travel costs.

In [4]:
all_rooftops = gpd.read_parquet(OUTPUT_DIR / 'samp_roofs_20241029_142812.parquet')

In [5]:
# identify isolated points to drop from sampling
all_rooftops['neighbors'] = count_neighbors_in_radius(all_rooftops)
all_rooftops['isolated'] = (all_rooftops['neighbors'] < 5)

In [7]:
# remove isolated points from the rooftop data
all_rooftops_wo_isolated = all_rooftops[~all_rooftops['isolated']]

# Sample 4 rooftops from each barangay

In [8]:
# sample up to 4 rooftops per barangay (or all if there are less than 4)
sampled_points = all_rooftops_wo_isolated.groupby('PSGC', group_keys=False).apply(lambda x: x.sample(n=min(4, x.shape[0])))

/var/folders/_b/2wf3gjnx42s4k_vs5n079mh40000gn/T/ipykernel_16011/1718080806.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_roofs = all_rooftops_wo_isolated.groupby('PSGC', group_keys=False).apply(lambda x: x.sample(n=min(4, x.shape[0])))


# Get nearest points on road 

# Save sample outputs

In [9]:
len(sampled_roofs)

1360